In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def load_dataset(csv_path):
    return pd.read_csv(csv_path)


In [ ]:
def preprocess_data(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['COBOL'])
    y = df['Python']
    return X, y, vectorizer

In [ ]:
def train_model(X, y):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model


In [ ]:
def predict_python_keyword(model, vectorizer, cobol_keyword):
    cobol_keyword_vectorized = vectorizer.transform([cobol_keyword])
    python_keyword = model.predict(cobol_keyword_vectorized)
    return python_keyword[0]

In [ ]:
def tokenize_and_predict(model, vectorizer, statement, df):
    multi_word_keywords = {
        'IS GREATER THAN': '>',
        'IS LESSER THAN': '<',
        'IS EQUAL TO': '==',
        'IS GREATER THAN AND EQUAL TO': '>=',
        'IS LESSER THAN AND EQUAL TO': '<='
    }

    tokens = word_tokenize(statement)

    predicted_keywords = []

    i = 0
    while i < len(tokens):
        if i < len(tokens) - 2:
            three_word_token = ' '.join(tokens[i:i+3]).upper()
            if three_word_token in multi_word_keywords:
                predicted_keywords.append(multi_word_keywords[three_word_token])
                i += 3
                continue

        if i < len(tokens) - 3:
            four_word_token = ' '.join(tokens[i:i+4]).upper()
            if four_word_token in multi_word_keywords:
                predicted_keywords.append(multi_word_keywords[four_word_token])
                i += 4
                continue

        if tokens[i].upper() in df['COBOL'].str.upper().values:
            predicted_keyword = predict_python_keyword(model, vectorizer, tokens[i].upper())
        else:
            predicted_keyword = tokens[i]

        predicted_keywords.append(predicted_keyword)
        i += 1

    return predicted_keywords


In [ ]:
# Function to format the Python code
def format_python_code(predicted_keywords):
    python_code_lines = []
    indent_level = 0
    indent_spaces = '  '

    skip_next = False
    display_text = ''

    for i, keyword in enumerate(predicted_keywords):
        if skip_next:
            skip_next = False
            continue

        # Increase indentation after these keywords
        if keyword in ['if', 'elif', 'else:', 'while', 'for']:
            line = indent_spaces * indent_level + keyword
            python_code_lines.append(line)
            indent_level += 1
        # Decrease indentation for end of blocks
        elif keyword in ['endif', 'endperform', 'endread', 'endwrite']:
            indent_level -= 1
            line = indent_spaces * indent_level + keyword
            python_code_lines.append(line)
        # Handle special cases for colons and newlines
        elif keyword == ':':
            python_code_lines[-1] += ':'
            python_code_lines.append('\n' + indent_spaces * indent_level)
        elif keyword == '':
            python_code_lines.append('')
        # Wrap 'print' statements and other statements that require brackets
        elif keyword == 'print':
            python_code_lines.append('\n' + indent_spaces * indent_level + 'print(' + display_text)
            display_text = ''  # Reset display_text after printing
        elif keyword.startswith('"') and keyword.endswith('"'):
            display_text = keyword  # Store the quoted text for the print statement
        else:
            # Handle 'THEN' keyword to avoid printing it
            if keyword == 'THEN':
                continue
            # Add space after the current keyword if it's not followed by a colon
            if i + 1 < len(predicted_keywords) and predicted_keywords[i + 1] != ':':
                python_code_lines.append(indent_spaces * indent_level + keyword + ' ')
            else:
                python_code_lines.append(indent_spaces * indent_level + keyword + ')')

    # Join all lines into a single string with newlines
    formatted_code = ''.join(python_code_lines)

    return formatted_code


In [ ]:
def main():
    csv_path = '/content/cobol_to_python_mapping.csv'
    df = load_dataset(csv_path)

    X, y, vectorizer = preprocess_data(df)

    model = train_model(X, y)

    while True:
        statement = input("Enter a statement to tokenize (or type 'exit' to quit): ")

        if statement.lower() == 'exit':
            print("Exiting the program.")
            break

        if any(char.islower() for char in statement):
            print("Error: Statement must contain only uppercase characters.")
            continue

        predicted_keywords = tokenize_and_predict(model, vectorizer, statement, df)

        python_code = format_python_code(predicted_keywords)

        print("Formatted Python Code:\n", python_code)




In [30]:
if __name__ == "__main__":
    main()

Formatted Python Code:
 
print(`` HELLO '')


KeyboardInterrupt: Interrupted by user